In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

from sklearn.linear_model import LinearRegression


- reference : [Deep Learning Prediction With Full Features](https://www.kaggle.com/p7476762/deep-learning-prediction-with-full-features#1.-Read-the-Data)

### 1. Read the Data

In [3]:
df_holidays = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
df_transaction = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
df_test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')

sample_submission = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')

In [4]:
# convert all 'data' columns to datetime pandas format
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format = '%Y-%m-%d')
df_oil['date'] = pd.to_datetime(df_oil['date'], format = '%Y-%m-%d')
df_transaction['date'] = pd.to_datetime(df_transaction['date'], format = '%Y-%m-%d')
df_train['date'] = pd.to_datetime(df_train['date'], format = '%Y-%m-%d')
df_test['date'] = pd.to_datetime(df_test['date'], format = '%Y-%m-%d')

### 2. Data Preprocessing

#### 2-1. Oil preprocessing and merge to train_data/test_data

- 결측값을 채우는 방법
    1. [df.fillna() 이용](https://rfriend.tistory.com/262)
    2. [선형회귀모형 이용](https://rfriend.tistory.com/636)
    3. [결측값 보간(interpolation)](https://rfriend.tistory.com/264)
    

In [5]:
#df_oil.set_index(df_oil.date).drop(columns = ['date'])
df_oil.dcoilwtico


In [6]:
ts = pd.Series(df_oil['dcoilwtico'])
ts.index = df_oil['date']
ts

In [7]:
ts = pd.Series(df_oil['dcoilwtico'])
ts.index = df_oil['date']
ts_fillna = ts.interpolate(method = 'time') # interporation
ts_fillna = ts_fillna.fillna(method = 'bfill') # backward

df_oil['dcoilwtico'] = list(ts_fillna)
df_oil

In [8]:
train_data_oil = pd.merge(df_train, df_oil, left_on = 'date', right_on = 'date', how = 'left')
test_data_oil = pd.merge(df_test, df_oil, left_on = 'date', right_on = 'date', how = 'left')

In [9]:
# train 
print(train_data_oil['dcoilwtico'].isna().sum())
ts = pd.Series(train_data_oil['dcoilwtico'])
ts.index = train_data_oil['date']
ts_fillna = ts.interpolate(method = 'time') # interporation
ts_fillna = ts_fillna.fillna(method = 'pad') # backward

train_data_oil['dcoilwtico'] = list(ts_fillna)
print(train_data_oil['dcoilwtico'].isna().sum())


In [10]:
# test
print(test_data_oil['dcoilwtico'].isna().sum())
ts = pd.Series(test_data_oil['dcoilwtico'])
ts.index = test_data_oil['date']
ts_fillna = ts.interpolate(method = 'time') # interporation
ts_fillna = ts_fillna.fillna(method = 'pad') # backward

test_data_oil['dcoilwtico'] = list(ts_fillna)
print(test_data_oil['dcoilwtico'].isna().sum())


#### 2-2 hoilday_events preprocessing and merge to train_data/test_data

In [11]:
# train
train_data_oil_holiday = pd.merge(train_data_oil, df_holidays, left_on = 'date', right_on = 'date', how = 'left')
#print(train_data_oil_holiday.isna().sum())
train_data_oil_holiday.fillna('Empty', inplace = True)
#print(train_data_oil_holiday.isna().sum())

# test
test_data_oil_holiday = pd.merge(test_data_oil, df_holidays, left_on = 'date', right_on = 'date', how = 'left')
#print(test_data_oil_holiday.isna().sum())
test_data_oil_holiday.fillna('Empty', inplace = True)
#print(test_data_oil_holiday.isna().sum())


#### 2-3 transactions preprocessing and merge to train_data/test_data

- train data
    1. store 별 평균
    2. store 별 보간법
    3. 회귀모형 추정값
- test data
    1. 시계열 예측으로 채워넣기
    
    
    
- 생각
    - test data를 예측하기 위해 사용한 모델로 예측된 값을 train data의 null 값에 대입

In [12]:
# merge train
train_data_oil_holiday_transactions = pd.merge(train_data_oil_holiday, df_transaction, left_on = ['date', 'store_nbr'], right_on = ['date', 'store_nbr'], how = 'left')
print(train_data_oil_holiday_transactions.isna().sum())

# merge test
test_data_oil_holiday_transactions = pd.merge(test_data_oil_holiday, df_transaction, left_on = ['date', 'store_nbr'], right_on = ['date', 'store_nbr'], how = 'left')
print(test_data_oil_holiday_transactions.isna().sum())

#### 2-4 Store preprocessing and merge to train_data/test_data

In [13]:
train_data = pd.merge(train_data_oil_holiday_transactions, df_stores, left_on = 'store_nbr', right_on = 'store_nbr', how = 'left')
test_data = pd.merge(test_data_oil_holiday_transactions, df_stores, left_on = 'store_nbr', right_on = 'store_nbr', how = 'left')

#### 2-5 year, month, trend, week columns merge to train_data/test_data

In [14]:
def split_year(time):
    time = str(time)
    return int(time.split('-')[0])

def split_month(time):
    time = str(time)
    return int(time.split('-')[1])

def split_trend(time):
    time = str(time)
    return int(time.split(' ')[0])

def weekend(date): # 토, 일
    import datetime
    weekend = []
    a = pd.to_datetime(date)
    for i in range(len(a)):
        if a.iloc[i].weekday() >= 5:
            weekend.append(1)
        else:
            weekend.append(0)
    return weekend

def weekday(date): # 월(0) ~ 일(6)
    import datetime
    weekday = []
    a = pd.to_datetime(date)
    for i in range(len(a)):
        weekday.append(a.iloc[i].weekday())
    return weekday

In [15]:
train_data['Year'] = train_data['date'].apply(split_year)
train_data['Month'] = train_data['date'].apply(split_month)
train_data['Weekend'] = weekend(train_data['date'])
train_data['Weekday'] = weekday(train_data['date'])
train_data['trend'] = (train_data['date'] - train_data['date'].min())
train_data['trend'] = train_data['trend'].apply(split_trend)
train_data.head()

In [16]:
test_data['Year'] = test_data['date'].apply(split_year)
test_data['Month'] = test_data['date'].apply(split_month)
test_data['Weekend'] = weekend(test_data['date'])
test_data['Weekday'] = weekday(test_data['date'])
test_data['trend'] = (test_data['date'] - train_data['date'].min())
test_data['trend'] = test_data['trend'].apply(split_trend)
test_data.head()

In [17]:
# 새해 열 추가
train_data['NewYearsDay'] = (train_data.date.dt.dayofyear == 1)
test_data['NewYearsDay'] = (test_data.date.dt.dayofyear == 1)

# 추가전 transactions predict model 성능
    # - train: 337
    # - valid: 351
# 추가후 transactions predict model 성능 - 똑같음
    # - train: 337
    # - valid: 351

In [18]:
train_data_notna = train_data[~train_data['transactions'].isna()]

In [19]:
# predict transaction

y = train_data_notna['transactions']
X1 = train_data_notna[['store_nbr', 'state', 'type_y', 'cluster', 'Year', 'Month', 'Weekend', 'Weekday', 'trend', 'dcoilwtico']]
y = y[X1.store_nbr != 52]
X1 = X1[X1.store_nbr != 52]


train_y = y[train_data_notna.date < '2016-08-16']
train_X1 = X1[train_data_notna.date < '2016-08-16']

valid_y = y[train_data_notna.date >= '2016-08-16']
valid_X1 = X1[train_data_notna.date >= '2016-08-16']


# store 52 이상하다.

In [20]:
#train_data[(train_data.store_nbr == 52) & (train_data.sales == 0)]

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

# preprocessing
preprocessor = ColumnTransformer(
        [('stand',StandardScaler(), ['dcoilwtico']),
        ('cat', OneHotEncoder(sparse = False, drop = 'first'), ['store_nbr', 'state', 'type_y'])],
        remainder = 'passthrough'

)

preprocessor.fit(train_X1)
enc_cat_features = preprocessor.named_transformers_['cat'].get_feature_names()
labels = np.concatenate([['dcoilwtico'], enc_cat_features, ['cluster', 'Year', 'Month', 'Weekend', 'Weekday', 'trend']])


train_X1_trans = pd.DataFrame(preprocessor.transform(train_X1), columns = labels)
valid_X1_trans = pd.DataFrame(preprocessor.transform(valid_X1), columns = labels)

model = LinearRegression(fit_intercept = False)
model.fit(train_X1_trans, train_y)

In [22]:
train_X1_pred = model.predict(train_X1_trans)
valid_X1_pred = model.predict(valid_X1_trans)

In [23]:
# 훈련데이터
np.sqrt(np.mean((train_X1_pred - train_y)**2))

In [24]:
# 검증데이터
np.sqrt(np.mean((valid_X1_pred - valid_y)**2))

In [25]:
# na가 아닌 데이터로 학습 후, 나머지 예측 (52포함)

# 데이터 준비
y = train_data_notna['transactions']
X = train_data_notna[['store_nbr', 'state', 'type_y', 'cluster', 'Year', 'Month', 'Weekend', 'Weekday', 'trend', 'dcoilwtico']]

full_train_y = train_data['transactions']
full_train_X = train_data[['store_nbr', 'state', 'type_y', 'cluster', 'Year', 'Month', 'Weekend', 'Weekday', 'trend', 'dcoilwtico']]

full_test_y = test_data['transactions']
full_test_X = test_data[['store_nbr', 'state', 'type_y', 'cluster', 'Year', 'Month', 'Weekend', 'Weekday', 'trend', 'dcoilwtico']]

# 전처리
preprocessor.fit(X)
enc_cat_features = preprocessor.named_transformers_['cat'].get_feature_names()
labels = np.concatenate([['dcoilwtico'], enc_cat_features, ['cluster', 'Year', 'Month', 'Weekend', 'Weekday', 'trend']])
X_trans = pd.DataFrame(preprocessor.transform(X), columns = labels)
full_train_X_trans = pd.DataFrame(preprocessor.transform(full_train_X), columns = labels)
full_test_X_trans = pd.DataFrame(preprocessor.transform(full_test_X), columns = labels)

# model
model.fit(X_trans, y)
train_X_pred = model.predict(full_train_X_trans)
test_X_pred = model.predict(full_test_X_trans)

In [26]:
# 값 대체하기
train_mask = (train_data['transactions'].isna())
train_data['transactions'][train_mask] = train_X_pred[train_mask]
test_data['transactions'] = test_X_pred # 전체가 na임

In [27]:
print(train_data.isna().sum())
print(test_data.isna().sum())


#### transaction null 값 채우기
1. store_nbr 52 인 데이터가 이상함.
2. 회귀모형이 얼마나 정확한 건지 잘 모르겠다.
3. 회귀, store 별 보간, 0 을 넣고 선형회귀모형 넣고 돌려서 가장 좋은 방법을 사용.

### additional notes
    - 2016년 4월 15일 에콰도르 지진. 이 때의 데이터를 어떻게 할 건지

In [28]:
np.save("./train_merge_data.npy", train_data)
np.save("./test_merge_data.npy", test_data)